**Bayesian network-(decision network)**graphical model, representing variables and their dependencies through DAG.

1) built from probability distribution and uses prob theory for:
 prediction
 anomaly detection,
 diagnostics 
 automated reasoning and insight
 time series prediction
 dec-making under uncertainity

2) COMPONENTS: 
    Directed Acyclic Graph: represent structure of dependency b/w variables
    Conditional Probability Tables: specify quantitative strength of those dependencies.

3) ELEMENTS:
    __nodes__: variables in the domain;observed or hidden
    __edges__: links b/w nodes representing dependencies
    __CPTs__: non-root node is associated cpt table which provides it with a    value(prob dist) given its parent nodes
    __parent-node__: nodes with edge pointing to another node
    __root-node__: node with no incoming edges(independen variabel)
    __leaf-node__: node with no outgoing edge, final nodes

    DiscreteBayesianNetwork('cause','result')
    TabularCPD(var='cause', var_card=2, values=[[no(0)],[yes(1)]])
*
from pgmpy.estimators import BayesianEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.estimators import ParameterEstimator

model = BayesianNetwork([('Burglary', 'Alarm'), ('Earthquake', 'Alarm')])
model.fit(data, estimator=BayesianEstimator, prior_type='BDeu')  # data is a pandas DataFrame
*


In [ ]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

#define struct
model = DiscreteBayesianNetwork([
    ('Burglary','Alarm'),('Quake','Alarm'),('Alarm','John'),('Alarm','Mary')])
#define table / values probability
cpd_burglary = TabularCPD(variable='Burglary',variable_card=2,values=[[0.999],[0.001]])
cpd_quake = TabularCPD(variable='Quake',variable_card=2,values=[[0.98],[0.02]])
#alarm dependent on quake or burg so values = 4
cpd_alarm = TabularCPD(variable='Alarm',variable_card=2,values=[[0.999,0.71,0.06,0.05],[0.001,0.29,0.94,0.95]],
                       evidence=['Burglary','Quake'],
                       evidence_card=[2,2])

cpd_john = TabularCPD(variable='John',variable_card=2,values=[[0.3,0.9],[0.7,0.1]], evidence=['Alarm'], evidence_card=[2])
cpd_mary = TabularCPD(variable='Mary',variable_card=2,values=[[0.2,0.99],[0.8,0.01]],evidence=['Alarm'],evidence_card=[2])

#add model
model.add_cpds(cpd_burglary,cpd_quake,cpd_alarm,cpd_john,cpd_mary)
#verify
assert model.check_model(),"model is incorrect"
#inference
inference = VariableElimination(model)
#query both john and mary call, burglary prob?
result = inference.query(variables=['Burglary'],evidence={'John':1,'Mary':1})
print(result)


+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.9999 |
+-------------+-----------------+
| Burglary(1) |          0.0001 |
+-------------+-----------------+


In [ ]:
#task 1
from fractions import Fraction

total_cards = 52
red_cards = 26  # Hearts + Diamonds
hearts = 13
diamonds = 13
spades = 13
clubs = 13

# Face cards: Jack, Queen, King in each suit (3 per suit)
face_cards_per_suit = 3
total_face_cards = 4 * face_cards_per_suit  # 12
diamond_face_cards = face_cards_per_suit
spade_face_cards = face_cards_per_suit
queen_cards = 4  

p_red = Fraction(red_cards, total_cards)
p_heart_given_red = Fraction(hearts, red_cards)
p_diamond_given_face = Fraction(diamond_face_cards, total_face_cards)
spade_or_queen = spade_face_cards + queen_cards - 1
p_spade_or_queen= Fraction(spade_or_queen, total_face_cards)

print("1. P(Red card):", p_red)
print("2. P(red heart):", p_heart_given_red)
print("3. P(diamond):", p_diamond_given_face)
print("4. P(quen or soade):", p_spade_or_queen)


1. P(Red card): 1/2
2. P(red heart): 1/2
3. P(diamond): 1/4
4. P(quen or soade): 1/2


In [ ]:
#task2
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([
    ('Intelligence','Grade'),
    ('StudyHours','Grade'),
    ('Difficulty','Grade'),
    ('Grade','Pass')
])

cpd_intelligence = TabularCPD(variable='Intelligence', variable_card=2, values=[[0.7],[0.3]],state_names={'Intelligence':['High','Low']})

cpd_studyhours = TabularCPD(variable='StudyHours',variable_card=2,values=[[0.6],[0.4]],state_names={'StudyHours':['Sufficient','Insufficient']})

cpd_difficulty = TabularCPD(variable='Difficulty',variable_card=2,values=[[0.4],[0.6]],state_names={'Difficulty':['Hard','Easy']})

cpd_grade = TabularCPD(variable='Grade',variable_card=3,values=[
    [0.8, 0.7, 0.6, 0.5, 0.7, 0.6, 0.4, 0.1],#a
    [0.15, 0.2, 0.3, 0.3, 0.2, 0.3, 0.4, 0.4],#b
    [0.05, 0.1, 0.1, 0.2, 0.1, 0.1, 0.2, 0.5]#c
    ],evidence=['Intelligence','StudyHours','Difficulty'], evidence_card=[2,2,2],state_names={
        'Grade':['A','B','C'],
        'Intelligence':['High','Low'],
        'StudyHours':['Sufficient','Insufficient'],
        'Difficulty':['Hard','Easy']
    })
cpd_pass = TabularCPD(variable='Pass',variable_card=2,values=[[0.05,0.2,0.5],[0.95,0.8,0.5]],evidence=['Grade'],evidence_card=[3],state_names={
    'Pass':['No','Yes'],
    'Grade':['A','B','C']
})

model.add_cpds(cpd_intelligence,cpd_studyhours,cpd_difficulty,cpd_grade,cpd_pass)

assert model.check_model(), "not correct"
inference = VariableElimination(model)
result1 = inference.query(variables=['Pass'],evidence={'StudyHours':'Sufficient','Difficulty':'Hard'})

result2 = inference.query(variables=['Intelligence'],
                         evidence={'Pass':'Yes'})
print(result1)
print(result2)

+-----------+-------------+
| Pass      |   phi(Pass) |
+===========+=============+
| Pass(No)  |      0.1040 |
+-----------+-------------+
| Pass(Yes) |      0.8960 |
+-----------+-------------+
+--------------------+---------------------+
| Intelligence       |   phi(Intelligence) |
+====================+=====================+
| Intelligence(High) |              0.7144 |
+--------------------+---------------------+
| Intelligence(Low)  |              0.2856 |
+--------------------+---------------------+


In [20]:
#task3
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = DiscreteBayesianNetwork([
    ('Disease','Fever'),
    ('Disease','Cough'),
    ('Disease','Fatigue'),
    ('Disease','Chills')
])

cpd_disease = TabularCPD(variable='Disease',
                         variable_card=2,values=[[0.3],[0.7]],state_names={'Disease': ['Flu','Cold']})
cpd_fever = TabularCPD(variable='Fever',variable_card=2,
                       values=[
                           [0.1,0.5],#no
                           [0.9,0.5]],#yes
                       evidence=['Disease'],
                       evidence_card=[2],
                       state_names={
                           'Fever':['No','Yes'],
                           'Disease':['Flu','Cold']})

cpd_cough = TabularCPD(variable='Cough',variable_card=2,
                       values=[
                           [0.2,0.4],#no
                           [0.8,0.6]],#yes
                       evidence=['Disease'],
                       evidence_card=[2],
                       state_names={
                           'Cough':['No','Yes'],
                           'Disease':['Flu','Cold']})

cpd_fatigue = TabularCPD(variable='Fatigue',variable_card=2,
                       values=[
                           [0.3,0.7],#no
                           [0.7,0.3]],#yes
                       evidence=['Disease'],
                       evidence_card=[2],
                       state_names={
                           'Fatigue':['No','Yes'],
                           'Disease':['Flu','Cold']})

cpd_chills= TabularCPD(variable='Chills',variable_card=2,
                       values=[
                           [0.4,0.6],#no
                           [0.6,0.4]],#yes
                       evidence=['Disease'],
                       evidence_card=[2],
                       state_names={
                           'Chills':['No','Yes'],
                           'Disease':['Flu','Cold']})

model.add_cpds(cpd_disease,cpd_fever,cpd_cough,cpd_fatigue,cpd_chills)

assert model.check_model(), "incorrect"
inference = VariableElimination(model)
result = inference.query(variables=['Disease'],evidence={
    'Fever':'Yes',
    'Cough':'Yes'
})
print(result)

result1  = inference.query(variables=['Disease'], evidence={'Fever':'Yes','Cough':'Yes','Chills':'Yes'})
print(result1)

result2 = inference.query(variables=['Fatigue'], evidence={
    'Disease':'Flu'
})
print(result2)

+---------------+----------------+
| Disease       |   phi(Disease) |
+===============+================+
| Disease(Flu)  |         0.5070 |
+---------------+----------------+
| Disease(Cold) |         0.4930 |
+---------------+----------------+
+---------------+----------------+
| Disease       |   phi(Disease) |
+===============+================+
| Disease(Flu)  |         0.6067 |
+---------------+----------------+
| Disease(Cold) |         0.3933 |
+---------------+----------------+
+--------------+----------------+
| Fatigue      |   phi(Fatigue) |
+==============+================+
| Fatigue(No)  |         0.3000 |
+--------------+----------------+
| Fatigue(Yes) |         0.7000 |
+--------------+----------------+


In [ ]:
#task4
import numpy as np

states = ['Sunny', 'Cloudy', 'Rainy']
state_index = {state: i for i, state in enumerate(states)}
transition_matrix = np.array([
    [0.6, 0.3, 0.1],  # From Sunny
    [0.3, 0.4, 0.3],  # From Cloudy
    [0.2, 0.4, 0.4]   # From Rainy
])

def simulate_weather(start_state='Sunny', days=10):
    current_state = state_index[start_state]
    weather_sequence = [start_state]
    for _ in range(days - 1):
        next_state = np.random.choice(states, p=transition_matrix[current_state])
        weather_sequence.append(next_state)
        current_state = state_index[next_state]
    return weather_sequence

num_simulations = 10000
count_at_least_3_rainy = 0

for _ in range(num_simulations):
    simulation = simulate_weather(start_state='Sunny', days=10)
    rainy_days = simulation.count('Rainy')
    if rainy_days >= 3:
        count_at_least_3_rainy += 1

probability = count_at_least_3_rainy / num_simulations
print(f"Estimated Probability of at least 3 rainy days in 10 days: {probability:.4f}")


Estimated Probability of at least 3 rainy days in 10 days: 0.3269
